# Build AI-driven workflows using Snowflake Cortex LLM Functions with pandas on Snowflake

pandas on Snowflake provides developers with a familiar pandas interface to interact with Snowflake. With pandas on Snowflake, users can work with large amounts of data through Snowpark pandas DataFrames or Series. [Snowflake Cortex LLM functions](https://docs.snowflake.com/en/user-guide/snowflake-cortex/llm-functions) can now be leveraged with pandas on Snowflake via the pandas [apply](https://docs.snowflake.com/en/developer-guide/snowpark/reference/python/latest/modin/pandas_api/modin.pandas.DataFrame.apply) method. This allows users to instantly access and apply custom LLM-powered functions trained by the worlds leading researchers to batches of text or documents stored in Snowpark DataFrames or Series. The [pandas on Snowflake integration with Snowflake Cortex LLM function](https://docs.snowflake.com/en/developer-guide/snowpark/python/pandas-on-snowflake#using-snowflake-cortex-llm-functions-with-snowpark-pandas) expands the capabilities for developers to build AI-driven workflows with pandas on Snowflake.

In this example we can classify menu items from a DataFrame into smaller categories using the Snowflake Cortex [CLASSIFY_TEXT](https://docs.snowflake.com/en/sql-reference/functions/classify_text-snowflake-cortex) LLM function.

## Adding Python Packages 🎒

Snowflake Notebooks comes pre-installed with common Python libraries. To add other packages, click on the `Packages` dropdown on the top right to add additional packages to your notebook.

Let's add the following packages: 
- `modin` 
- `snowflake-ml-python`
- `s3fs`

In [ ]:
import streamlit as st
import modin.pandas as pd
import snowflake.snowpark.modin.plugin

## Connecting to Snowflake 

To work with your data in Snowflake, you need to first get a session variable to connect to Snowflake. Since you are already logged in to Snowflake Notebook, you can get your session variable directly through the active notebook session. The session variable is the entrypoint that gives you access to using Snowflake's Python API, including Snowpark.

In [ ]:
from snowflake.snowpark.context import get_active_session
session = get_active_session()

## Bringing data into Snowflake
You can use pandas on Snowflake to load in [CSV](https://docs.snowflake.com/en/developer-guide/snowpark/reference/python/latest/modin/pandas_api/modin.pandas.read_csv#modin.pandas.read_csv), [Parquet](https://docs.snowflake.com/en/developer-guide/snowpark/reference/python/latest/modin/pandas_api/modin.pandas.read_parquet#modin.pandas.read_parquet), and [Excel](https://docs.snowflake.com/en/developer-guide/snowpark/reference/python/latest/modin/pandas_api/modin.pandas.read_excel#modin.pandas.read_excel) from stage or local file location. Here is the full list of [I/O functionalities supported](https://docs.snowflake.com/en/developer-guide/snowpark/reference/python/1.30.0/modin/io).


First let's create a external stage and upload the CSV file. 

In [ ]:
CREATE OR REPLACE STAGE FROSTBYTES
    URL = 's3://sfquickstarts/frostbyte_tastybytes/';

In [ ]:
menu_item = pd.read_csv("@frostbytes/analytics/menu_item_aggregate_v.csv")

In [ ]:
menu_item.head()

## Data Cleaning and Transformation

Now let's clean up the data by filtering to only records in January 2023.


In [ ]:
menu_item["DATE"] = pd.to_datetime(menu_item["DATE"])
filtered_menu_item = menu_item[(menu_item["DATE"]>'2023-01-01')&(menu_item["DATE"]<'2023-02-01')]
st.markdown(f'There are {len(menu_item)} rows in the full dataset. After filtering, there are {len(filtered_menu_item)} rows in this daterange.' )

In [ ]:
st.markdown(f'''There are {len(filtered_menu_item["MENU_ITEM_NAME"].unique())} different menu items. \n
That's a lot of different items! Let's see how we can group them into fewer categories.''')

Next, to clean up the data further, we want to to classify menu items into a smaller number of categories. We can do that using Snowflake's Cortex LLM functions [CLASSIFY_TEXT](https://docs.snowflake.com/en/sql-reference/functions/classify_text-snowflake-cortex). You can use Snowflake Cortex LLM functions via the Snowpark pandas apply function, see examples [here](https://docs.snowflake.com/en/developer-guide/snowpark/python/pandas-on-snowflake#using-snowflake-cortex-llm-functions-with-snowpark-pandas).

In [ ]:
from snowflake.cortex import ClassifyText
filtered_menu_item["MENU_ITEM_CATEGORY"] = filtered_menu_item["MENU_ITEM_NAME"].apply(ClassifyText, categories=["Meal","Dessert","Drinks"])

In [ ]:
filtered_menu_item["MENU_ITEM_CATEGORY"]

Now let's extract the `label` field from the dictionary in the column.

In [ ]:
filtered_menu_item["MENU_ITEM_LABEL"] = filtered_menu_item["MENU_ITEM_CATEGORY"].apply(lambda x: x.get('label'))

In [ ]:
filtered_menu_item["MENU_ITEM_LABEL"]

For more examples of other Cortex functions you can use with pandas on Snowflake, visit our pandas on Snowflake [developer guide](https://docs.snowflake.com/en/developer-guide/snowpark/python/pandas-on-snowflake#using-snowflake-cortex-llm-functions-with-snowpark-pandas) and Snowflake Cortex [user guide](https://docs.snowflake.com/en/user-guide/snowflake-cortex/llm-functions) to learn more.